In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from pathlib import Path
from google.colab import drive

drive.mount("/content/drive")

spotify_df = pd.read_csv(Path("/content/drive/My Drive/spotify-2023_updated.csv"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
spotify_df = spotify_df.drop(["track_name", "artist_name", "in_spotify_charts", "in_apple_charts"], axis=1)

In [3]:
df = spotify_df.drop("mode", axis = 1)

dummies_df = pd.get_dummies(spotify_df["mode"])

final_df = pd.concat([df, dummies_df], axis=1)
final_df.head()

,artist_count,released_year,released_month,released_day,streams,bpm,danceability_pct,valence_pct,energy_pct,acousticness_pct,instrumentalness_pct,liveness_pct,speechiness_pct,Major,Minor
0,2,2023,7,14,141381703,125,80,89,83,31,0,8,4,1,0
1,1,2023,3,23,133716286,92,71,61,74,7,0,10,4,1,0
2,1,2023,6,30,140003974,138,51,32,53,17,0,31,6,1,0
3,1,2019,8,23,800840817,170,55,58,72,11,0,11,15,1,0
4,1,2023,5,18,303236322,144,65,23,80,14,63,11,6,0,1


In [4]:
streaming_bin = [0, 250000000, 3803895074]
label = [int("0"), int("1")]

final_df["song_status"] = pd.cut(final_df["streams"], streaming_bin, labels=label)
final_df["song_status"] = final_df["song_status"].astype(int)
final_df.dtypes

# 0 = bad song, 1 = good song

artist_count            int64
released_year           int64
released_month          int64
released_day            int64
streams                 int64
bpm                     int64
danceability_pct        int64
valence_pct             int64
energy_pct              int64
acousticness_pct        int64
instrumentalness_pct    int64
liveness_pct            int64
speechiness_pct         int64
Major                   uint8
Minor                   uint8
song_status             int64
dtype: object

In [5]:
# splitting our data into target and features

# defining features
X = final_df.drop("song_status", axis=1)

# defining target
y = final_df["song_status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# scaling the data

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# make our model

number_input_features = len(X_train.iloc[0])
layer1=50
layer2=15

model = tf.keras.models.Sequential()

# first layer

model.add(
    tf.keras.layers.Dense(units=layer1, input_dim=number_input_features, activation="relu")
)

# second layer

model.add(
    tf.keras.layers.Dense(units=layer2, activation="relu")
)

# output layer

model.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                800       
                                                                 
 dense_4 (Dense)             (None, 15)                765       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1581 (6.18 KB)
Trainable params: 1581 (6.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
fit_model = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
23/23 [==============================] - 3s 8ms/step - loss: 0.7179 - accuracy: 0.4748
Epoch 2/100
23/23 [==============================] - 0s 10ms/step - loss: 0.6542 - accuracy: 0.6331
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6085 - accuracy: 0.7045
Epoch 4/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5648 - accuracy: 0.7437
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5202 - accuracy: 0.7829
Epoch 6/100
23/23 [==============================] - 0s 7ms/step - loss: 0.4790 - accuracy: 0.8151
Epoch 7/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4401 - accuracy: 0.8347
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4023 - accuracy: 0.8557
Epoch 9/100
23/23 [==============================] - 0s 4ms/step - loss: 0.3688 - accuracy: 0.8599
Epoch 10/100
23/23 [==============================] - 0s 8ms/step - loss: 0.3390 - accuracy: 0.8655
Epoch 11

In [11]:
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.0883 - accuracy: 0.9623 - 187ms/epoch - 23ms/step
Loss: 0.0882761999964714, Accuracy: 0.9623430967330933


In [12]:
prediction = model.predict(X_test_scaled)

8/8 [==============================] - 0s 2ms/step


In [21]:
prediction[0]

array([1.], dtype=float32)